In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
fid = drive.ListFile({'q':"title='fish_data.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('fish_data.zip')
f.keys()
!unzip fish_data.zip

Archive:  fish_data.zip
   creating: fish data/train/
   creating: fish data/train/Bowal/
  inflating: fish data/train/Bowal/download - Copy.jpeg  
  inflating: fish data/train/Bowal/download (1) - Copy.jpeg  
  inflating: fish data/train/Bowal/download (1).jpeg  
  inflating: fish data/train/Bowal/download (3) - Copy.jpeg  
  inflating: fish data/train/Bowal/download (3).jpeg  
  inflating: fish data/train/Bowal/download (4) - Copy.jpeg  
  inflating: fish data/train/Bowal/download (4).jpeg  
  inflating: fish data/train/Bowal/download (5) - Copy.jpeg  
  inflating: fish data/train/Bowal/download (5).jpeg  
  inflating: fish data/train/Bowal/download.jpeg  
  inflating: fish data/train/Bowal/images (1) - Copy.jpeg  
  inflating: fish data/train/Bowal/images (1).jpeg  
  inflating: fish data/train/Bowal/images (10) - Copy.jpeg  
  inflating: fish data/train/Bowal/images (10).jpeg  
  inflating: fish data/train/Bowal/images (11) - Copy.jpeg  
  inflating: fish data/train/Bowal/images (1

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
import cv2
import os
from tqdm import tqdm

folder_dir = '/content/fish data/train'
SIZE = (75, 75) 
DOWNSAMPLE_RATIO = 4
JPEG_QUALITY = 60

# get the total number of files to process
total_files = sum(len(files) for _, _, files in os.walk(folder_dir))

# use tqdm to display a progress bar
with tqdm(total=total_files, desc="Processing Images") as pbar:
    for folder in os.listdir(folder_dir):
        for file in os.listdir(os.path.join(folder_dir, folder)):
            image_path = os.path.join(folder_dir, folder, file)
            # check if the file has read permission
            if not os.access(image_path, os.R_OK):
                print(f"No read permission for: {image_path}")
                continue
            # read the image
            img = cv2.imread(image_path)
            if img is None:
                print(f"Failed to read image: {image_path}")
                continue
            # check if the image is already in JPEG format
            if os.path.splitext(file)[-1].lower() in ['.jpg', '.jpeg']:
                img_resized = cv2.resize(img, SIZE)
                # write the resized image
                success = cv2.imwrite(image_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, JPEG_QUALITY])
                if not success:
                    print(f"Failed to write image: {image_path}")
            else:
                # convert the image to JPEG format
                new_image_path = os.path.splitext(image_path)[0] + '.jpg'
                img_resized = cv2.resize(img, SIZE)
                # write the resized image in JPEG format
                success = cv2.imwrite(new_image_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, JPEG_QUALITY])
                if not success:
                    print(f"Failed to write image: {new_image_path}")
                else:
                    # delete the original image and replace it with the new JPEG image
                    os.remove(image_path)
                    os.rename(new_image_path, image_path)
            pbar.update(1) # increment the progress bar

Processing Images: 100%|██████████| 1882/1882 [00:00<00:00, 2699.69it/s]


In [ ]:
# Add a new top (classification) layer for fish classification
from tensorflow import keras
from tensorflow.keras import layers

inception_base = keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(75, 75, 3)
)

for layer in inception_base.layers:
    layer.trainable = False

model = keras.Sequential([
    inception_base,
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(9, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Data preprocessing
train_data_dir = '/content/fish data/train'
validation_data_dir = '/content/fish data/validation'
batch_size = 32
img_height = 75
img_width = 75

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 1736 images belonging to 9 classes.
Found 438 images belonging to 9 classes.


In [ ]:
# Train the model
epochs = 30
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/30
54/54 [==============================] - 13s 151ms/step - loss: 2.3521 - accuracy: 0.2412 - val_loss: 1.5642 - val_accuracy: 0.4591
Epoch 2/30
54/54 [==============================] - 7s 134ms/step - loss: 1.6235 - accuracy: 0.4454 - val_loss: 1.1716 - val_accuracy: 0.6082
Epoch 3/30
54/54 [==============================] - 7s 122ms/step - loss: 1.3169 - accuracy: 0.5311 - val_loss: 0.9537 - val_accuracy: 0.6947
Epoch 4/30
54/54 [==============================] - 6s 105ms/step - loss: 1.1963 - accuracy: 0.6062 - val_loss: 0.8846 - val_accuracy: 0.7188
Epoch 5/30
54/54 [==============================] - 5s 98ms/step - loss: 1.1080 - accuracy: 0.6068 - val_loss: 0.7965 - val_accuracy: 0.7572
Epoch 6/30
54/54 [==============================] - 6s 112ms/step - loss: 1.0187 - accuracy: 0.6602 - val_loss: 0.8508 - val_accuracy: 0.7500
Epoch 7/30
54/54 [==============================] - 6s 117ms/step - loss: 0.9898 - accuracy: 0.6626 - val_loss: 0.7088 - val_accuracy: 0.7909
Epoch 

In [ ]:
#FINETUNING
model.trainable = True
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    initial_epoch = 10)

Epoch 11/20
54/54 [==============================] - 6s 102ms/step - loss: 0.6644 - accuracy: 0.7793 - val_loss: 0.4290 - val_accuracy: 0.8702
Epoch 12/20
54/54 [==============================] - 6s 109ms/step - loss: 0.6213 - accuracy: 0.7999 - val_loss: 0.4489 - val_accuracy: 0.8678
Epoch 13/20
54/54 [==============================] - 6s 112ms/step - loss: 0.6367 - accuracy: 0.7969 - val_loss: 0.4127 - val_accuracy: 0.8750
Epoch 14/20
54/54 [==============================] - 6s 116ms/step - loss: 0.6406 - accuracy: 0.7852 - val_loss: 0.4068 - val_accuracy: 0.8630
Epoch 15/20
54/54 [==============================] - 6s 117ms/step - loss: 0.6107 - accuracy: 0.8099 - val_loss: 0.4214 - val_accuracy: 0.8846
Epoch 16/20
54/54 [==============================] - 6s 113ms/step - loss: 0.6212 - accuracy: 0.7946 - val_loss: 0.4576 - val_accuracy: 0.8702
Epoch 17/20
54/54 [==============================] - 5s 95ms/step - loss: 0.6158 - accuracy: 0.7946 - val_loss: 0.5027 - val_accuracy: 0.8486


In [ ]:
for layer in inception_base.layers[:-10]:
    layer.trainable = False
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    initial_epoch = 20)

Epoch 21/30
54/54 [==============================] - 7s 129ms/step - loss: 0.6196 - accuracy: 0.7993 - val_loss: 0.4459 - val_accuracy: 0.8582
Epoch 22/30
54/54 [==============================] - 6s 116ms/step - loss: 0.6112 - accuracy: 0.7987 - val_loss: 0.4026 - val_accuracy: 0.8822
Epoch 23/30
54/54 [==============================] - 6s 116ms/step - loss: 0.6299 - accuracy: 0.7899 - val_loss: 0.4125 - val_accuracy: 0.8822
Epoch 24/30
54/54 [==============================] - 6s 110ms/step - loss: 0.5907 - accuracy: 0.8063 - val_loss: 0.3906 - val_accuracy: 0.8822
Epoch 25/30
54/54 [==============================] - 6s 115ms/step - loss: 0.5935 - accuracy: 0.7928 - val_loss: 0.4195 - val_accuracy: 0.8678
Epoch 26/30
54/54 [==============================] - 6s 111ms/step - loss: 0.6799 - accuracy: 0.7735 - val_loss: 0.3851 - val_accuracy: 0.8726
Epoch 27/30
54/54 [==============================] - 7s 125ms/step - loss: 0.5789 - accuracy: 0.7958 - val_loss: 0.3488 - val_accuracy: 0.9014

In [ ]:
for layer in inception_base.layers:
    layer.trainable = False
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    initial_epoch = 30)

Epoch 31/50
54/54 [==============================] - 6s 119ms/step - loss: 0.5892 - accuracy: 0.8093 - val_loss: 0.4066 - val_accuracy: 0.8654
Epoch 32/50
54/54 [==============================] - 7s 129ms/step - loss: 0.5768 - accuracy: 0.7969 - val_loss: 0.3608 - val_accuracy: 0.8894
Epoch 33/50
54/54 [==============================] - 5s 94ms/step - loss: 0.6170 - accuracy: 0.7975 - val_loss: 0.3723 - val_accuracy: 0.8942
Epoch 34/50
54/54 [==============================] - 6s 111ms/step - loss: 0.5627 - accuracy: 0.8075 - val_loss: 0.4233 - val_accuracy: 0.8822
Epoch 35/50
54/54 [==============================] - 5s 97ms/step - loss: 0.5910 - accuracy: 0.8034 - val_loss: 0.3788 - val_accuracy: 0.8798
Epoch 36/50
54/54 [==============================] - 7s 134ms/step - loss: 0.5610 - accuracy: 0.8116 - val_loss: 0.4000 - val_accuracy: 0.8774
Epoch 37/50
54/54 [==============================] - 6s 119ms/step - loss: 0.5913 - accuracy: 0.8099 - val_loss: 0.3587 - val_accuracy: 0.8990
E

In [ ]:
# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(validation_generator)
print("Validation accuracy:", val_acc)

14/14 [==============================] - 2s 112ms/step - loss: 0.3212 - accuracy: 0.8995
Validation accuracy: 0.8995434045791626


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# set the path to the validation directory
val_dir = '/content/fish data/validation'

# create an ImageDataGenerator to preprocess the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

# set batch size to 1 for testing
batch_size = 1

# use the validation directory to create a validation data generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# make predictions on the validation data
predictions = model.predict(val_generator, steps=val_generator.n)

# get the actual labels from the validation data generator
y_true = val_generator.classes

# get the predicted labels from the model predictions
y_pred = np.argmax(predictions, axis=1)

# get the class labels from the validation data generator
class_labels = list(val_generator.class_indices.keys())

# print the actual and predicted classes for the first 15 images
acc = 0
for i in range(100):
    actual_class = class_labels[y_true[i]]
    predicted_class = class_labels[y_pred[i]]
    if actual_class == predicted_class:
      acc+=1
    print(f"Actual class: {actual_class}, Predicted class: {predicted_class}")

print("Acc:",acc,"%")


Found 438 images belonging to 9 classes.
438/438 [==============================] - 4s 10ms/step
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Tangra
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Tangra
Actual class: Bowal, Predicted class: Tangra
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Tangra
Actual class: Bowal, Predicted class: Tangra
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowal, Predicted class: Bowal
Actual class: Bowa